<h1>Multiple Linear Regression</h1>
<h2>A Cereal Success:</h2>
<br>
Let's suppose that we are a cereal company trying to bring our new cereal <i>Rutgos</i> to market. However, we have determined that we want our cereal to be a healthy, yet delicious alternative to many of the market brand cereals around.
<br>
We have collected nutrition data on 80 of the most popular cereal brands and have collected and averaged the consumer rating for each of these cereals.
<br>
We want to see what the <b>predicted rating</b> of our cereal will be <b>given the nutritional constraints<b> we have imposed.
<br>
<img src = "https://github.com/rucogsciclub/Data_Science_Bootcamp/raw/master/Week_3/resources/cereal.png" width="200"></img>

In [ ]:
import numpy as np
import statsmodels.api as sm
import matplotlib as matplot
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import pandas as pd
import sklearn
import seaborn as sns
plt.rcParams["figure.figsize"] = (16,8)

# Set the style for all graphs (more options can be found: https://python-graph-gallery.com/104-seaborn-themes/)
sns.set_style('whitegrid')

In [ ]:
# Loading in the csv with our cereal data as a pandas DataFrame
# Original kaggle link: https://www.kaggle.com/crawford/80-cereals
cereals_df_raw = pd.read_csv("https://github.com/rucogsciclub/Data_Science_Bootcamp/raw/master/Week_3/data/cereal.csv")

In [ ]:
# Getting general info on our data
cereals_df_raw.info()

In [ ]:
# Looking at the first 10 rows
cereals_df_raw.head(10)

<h2>Cleaning Up The Data:</h2>
<br>
We noticed that there were some negative values for some of the nutrition facts, so we set those to 0.
<br>
Additionally, we noticed that the weight (in ounces) of the cereal was variable across different cereals, which would affect all of the other nutrition facts. So we decided to normalize all of the nutrition facts, by figuring them all out in terms of 1 ounce of weight. To do this, we simply divided by weight.

In [ ]:
# Takes all negative values and converts them to 0
def fix_negatives(n):
  if (type(n) is str):
    n_clean = n.replace('-','').replace('.','')
  else:
    n_clean = n
  if (type(n_clean) is int or type(n_clean) is float) or (type(n_clean) is str and n_clean.isnumeric()):
    return max(0, float(n))
  else:
    return n

# Fixing negatives
cereals_df = cereals_df_raw.applymap(fix_negatives)


# Choosing features to normalize by weight
features_to_normalize = ['calories', 'protein', 'fat', 'sodium', 'fiber', 'carbo', 'sugars', 'potass', 'vitamins', 'cups']
# For each feature, normalize it by weight
for feature in features_to_normalize:
  cereals_df[feature] = cereals_df[feature] / cereals_df['weight']
# Now normalize the weight column
cereals_df['weight'] = cereals_df['weight'] / cereals_df['weight']

cereals_df

<h2>Isolating Features</h2>
<br>
First, we want to isolate our features (aka independent variables) as we did last week. These features should be chosen as the nutrition facts of interest.

In [ ]:
features = ['calories', 'potass', 'sodium', 'protein', 'fat','fiber', 'carbo', 'sugars']
X = cereals_df[features]

<h2>Some Exploratory Data Analysis:</h2>

Similar to last week, let's use box plots to see the distribution of our features across all cereals.

In [ ]:
#This looks shitty! How do we fix (shown in next cell)
sns.boxplot(x="variable", y="value", data=pd.melt(X))
plt.show()

In [ ]:
# This specifies that we want one row of plots with two columns.
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,10))

# Here we choose the smaller values to be represented in one column
smallervalues = ['protein', 'fat', 'fiber', 'carbo', 'sugars']
X_small = X[smallervalues]
sns.boxplot(x='variable', y='value', ax=ax1, data=pd.melt(X_small))

# Here we choose the the larger values to be represented in the other column
largervalues = ['calories', 'potass', 'sodium']
X_large = X[largervalues]
sns.boxplot(x='variable', y='value', ax=ax2, data=pd.melt(X_large)) # And see just strikers in the second

# Setting the title of our figure
fig.suptitle('Distribution of Each Nutritional Component', fontsize=16)

plt.show()


<h3>Questions:</h3>
<ol type="1">
    <li>Based on the distributions above, what nutritional component has the widest range of values?</li>
    <li>Which nutritional component has the smallest range, and why might this be?</li>
    <li>What might this tell us about which nutritional components we should consider improving upon and which ones are pretty set in stone?</li>
</ol>


Let's also take a look at the distribution of ratings across all of the cereals as well to see what we're up against...

In [ ]:
sns.distplot(cereals_df['rating'], color="m")
plt.show()

<h3>Question:</h3>
Does this distribution match up to our general expectation of how consumers would rate a broad number of different cereals?

<h2>The Curse of Multicollinearity</h2>
<br>
We now come upon having to set up our model. We have chosen and performed a preliminary analysis of our features and the target (rating).
<br>
However, our linear regression model will be making the assumption that our features are <b>independent</b> of one another. Simply, this means that any change in any one of our features should ideally not have an effect on any of the other features.
<br>
When we have one feature depending on another, we have <b>multicollinearity</b>.

<h2> Correlation Matrix </h2>
<br>
Now, let's take a look at the correlation matrix of our features. 
<br>
This will tell us how correlated each feature is to each other feature in our data.

In [ ]:
correlation_matrix = X.corr().round(2)
fig, ax = plt.subplots(figsize=(20,12))   
sns.heatmap(data=correlation_matrix, annot=True)
plt.show()

<h3>Questions:</h3>
<ol type="1">
    <li>Based on this matrix, do relationships between features match up with our expectations for nutritional facts?</li>
    <li>What are some relationships in the data that don't match up with our expectations?</li>
    <li>Are correlations enough to tell us whether we have multicollinearity for any of our features?</li>
</ol>

<h2>Variance Inflation Factor (VIF)</h2>
<br>
As we have seen, we can start to see the inklings of multicollinearity arise through analysis of the correlation matrix. However, what we want to see is not correlation, but rather how much one variable <i>depends</i> on another.
<br>
One way of formulating the notion of dependence of one variable on one or multiple other variables is precisely what we have been doing all along: <b>linear regression</b>.
<h3>Question:</h3> 
How can we use linear regression to verify whether one of our independent variables is dependent on the rest?

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF

vif = pd.DataFrame()
vif["VIF Factor"] = [VIF(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif

We can get this <b>VIF factor</b> by first doing linear regression with our chosen feature:
<br>
$X_i = \alpha_0 + \alpha_1 X_1 + \alpha_2 X_2 + ...$, where all the subscripts on the right hand side are not equal to $i$
<br>
Or, in other words we are doing linear regression with our <i>chosen feature</i> as the <i>dependent variable</i> and the <i>rest of our features</i> as the <i>independent variables</i>
<br>
We then compute the $R^2$ (<b>R-squared</b>) of this regression, which will tell us how well the change in the rest of our features explain the change in our chosen feature's behavior.
<br>
Then we compute the VIF factor as
<br>
$VIF = \frac{1}{1-R^2}$
<br>
The VIF factor tells us how high the dependence is between a chosen feature and the rest of the features.
<br>
<h3>Question:</h3>
    What happens to VIF when $R^2$ is high and what happens when it is low? Why?
<br><br>
As a general rule, we want to <i>discard or modify</i> those features which <i>exceed a VIF of 10</i>.

In [ ]:
X_old = X

In [ ]:
X = X.drop(labels = 'calories', axis = 1)
vif = pd.DataFrame()
vif["VIF Factor"] = [VIF(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif

It makes sense that calories had such a high VIF since fat, protein, carbs, and sugars all have a considerable impact on the calorie count so for any change in one of these features, you will expect a corresponding dependent change in calories.

In [ ]:
X = X.drop(labels = 'potass', axis = 1)
vif = pd.DataFrame()
vif["VIF Factor"] = [VIF(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif

In a less intuitive example, potassium was also found to have a pretty high VIF factor. As you can see, sometimes relationships in the data may arise, which we don't have an intuitive explanation for, but still influence our modeling approach.
<br><br>
Also, notice generally that VIF factors for all our features change each time we remove a feature.
<h3>Question:</h3>
Why does our VIF factor for all our features change each time we add, remove, or modify a feature?

<h2>Running Multiple Linear Regression</h2>
<br>
Now, we have arrived at the point where we can finally try to perform multiple linear regression on our model. Remember, the general formula is:
<br>
$Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + ...$, where each $X_i$ stands for one of our features and each $\beta_i$ represents how much a unit increase in $X_i$ contributes to a change in $Y$

In [ ]:
X_reg = sm.add_constant(X)
Y = cereals_df['rating']
model = sm.OLS(Y,X_reg).fit()
print(model.summary())

<h2>A Quick Side Note on Confidence Interval</h2>
<br>
When looking at our model, we may want to understand which of our coefficient choices is <i>actually</i> "significant."
<br>
The first thing to note is the format of the confidence interval, which is that it starts at 0.025 and ends at .975. If you subtract the left endpoint from the right endpoint you get a (.975 - .025) = .95 or 95% confidence interval. 95% is the default interval usually chosen.
<br>
What a confidence interval allows to say is that if 0 is found nowhere inside of it, then there is sufficient evidence to suggest that our regression coefficient is not equal to 0 i.e. "significant" at this confidence interval, e.g. when this coefficient takes on any of the values in this confidence interval, we consider it "significant".
<br>
<h4>Tidbit</h4>
More specifically, this tells us is that for this particular regression coefficient $\beta_i$, if we had an infinite number of cereal brands and each time we pulled out a sample of 80 cereals, they were distributed the same as these 80 cereals, 95% of the time it will be in our confidence interval.

<h2>Using Domain Knowledge and Modifying Features</h2>
<br>
We, as data scientists, have been doing a pretty good job. We have removed all the obviously apparent features that exhibited what was an excessively high VIF factor so that our regression is meaningful. Additionally, we saw that we got quite a good fit to the data.
<br>
<br>
However, we as data scientists also need to remain in constant consultation with specialists in the domain of the data we are working with. If you recall, our VIF factor for carbs, while not excessively high (above 10) was still relatively high compared to the other features.
<br>
To anyone experienced in working with the relationships between nutritional components, this fact may become apparent to you now. Just like calories, <i>carbs</i> are an <i>aggregate of multiple other nutritional components</i>.
<br>
<br>
In the case of calories, it was difficult to determine which other nutritional components were contributing how much to calories so we removed the feature. However, with carbohydrates, it is easy to modify this feature: In our case, the only major contributing component to carbohydrates from all our other features is <i>sugar</i> and sugar has a clear 1 to 1 relationship with carbohydrates i.e. 1 carb = 1 gram of sugar.
<br>
We can then utilize the domain knowledge we have of nutrition to ensure our features are even more independent of one another by subtracting sugars from carbs!

In [ ]:
print("Min carbs before:", min(X['carbo']), ", Max carbs before:", max(X['carbo']))
# Subtract sugars from carbs to avoid multicollinearity
X['carbo'] = X['carbo'] - X['sugars']
# Why might the following code demonstrate an issue in this concept?
print("Min carbs after:", min(X['carbo']), ", Max carbs after:", max(X['carbo']))

It is possible that subtracting the sugars may not be the correct move here. It is could be that this was already done when the data was collected since we ended up with negative numbers after subtraction. However, let's rerun our VIF factor analysis and regression and see what happens...

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [VIF(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif

Notice how the VIF for carbo has gone down quite significantly!
<br>
<h3>Open Questions:</h3>
<ol type="1">
    <li>Do you think it was a good choice to subtract sugars from carbs? Why or why not?
    <li>Which other feature's VIF looks suspiciously high and why might that be?</li>

In [ ]:
X_reg = sm.add_constant(X)
model = sm.OLS(Y,X_reg).fit()
print(model.summary())

<h3>Question</h3>
<br>
If we claim that rating is actually a metric for the overall enjoyment of the cereal, are there any regression coefficients that may suggest otherwise? If so, could this be due to multicollinearity or some other factor?
<br>
<br>
Now, let's compare that to a regression model using our original features, which exhibited multicollinearity...

In [ ]:
X_reg = sm.add_constant(X_old)
model_multicol = sm.OLS(Y,X_reg).fit()
print(model_multicol.summary())

<h3>Questions:</h3>
<ol type="1">
    <li>Did our model actually "improve" by removing our features that caused multicollinearity?</li>
    <li>Does a higher $R^2$ always suggest a "better" model? If not, then what does $R^2$ <i>really</i> represent?</li>

<h2>Making Rutgos</h2>
<br>
Now, since we want to emphasize the nutritious aspect of our cereal, we want our fats, our sugars, our carbs, and our sodium to be at the bottom of the distribution of cereals, while having a high protein and fiber content. We will ignore the number of calories and the amount of potassium since we removed those variables from our model earlier (however, since we saw these as the cause for multicollinearity, we should be able to derive these facts from our other nutritional information in the real world). Let's take a look at the distribution of our finalized features using boxplots once again to see where our nutritional information should lie:

In [ ]:
# This specifies that we want one row of plots with two columns.
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,10))

# Here we choose the smaller values to be represented in one column
smallervalues = ['protein', 'fat', 'fiber', 'carbo', 'sugars']
X_small = X[smallervalues]
sns.boxplot(x='variable', y='value', ax=ax1, data=pd.melt(X_small))

# Here we choose the the larger values to be represented in the other column
largervalues = ['sodium']
X_large = X[largervalues]
sns.boxplot(x='variable', y='value', ax=ax2, data=pd.melt(X_large)) # And see just strikers in the second

# Setting the title of our figure
fig.suptitle('Distribution of Each Nutritional Component', fontsize=16)

plt.show()

In [ ]:
rutgos_nutrition = {'protein':5, 'fat':10, 'fiber':500, 'carbo':25, 'sugars':1, 'sodium':550}
X_rutgos = pd.DataFrame(columns=X.columns)
X_rutgos.loc[0] = [rutgos_nutrition['sodium'], rutgos_nutrition['protein'], rutgos_nutrition['fat'], 
                      rutgos_nutrition['fiber'], rutgos_nutrition['carbo'], rutgos_nutrition['sugars']]
X_rutgos_pred = sm.add_constant(X_rutgos, has_constant='add')
prediction = model.predict(X_rutgos_pred)
print(prediction.loc[0])

Wow! It appears that a nutritious cereal may actually have a place in this market, or does it?
<h3>Questions</h3>
<ol type="1">
    <li>Does it seem reasonable that a cereal with no sugars, no fats, no carbs, and no salt can <i>actually</i> taste good?</li>
    <li>Is our model of linear regression the best way to encode the relationships between each of these nutritional components and the ranking of the cereal or is this too much of a simplification?</li>
    <li>What happens as we vary each of Rutgos's nutrition facts? What does this tell us about how linear regression works?</li>